In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os
import time

In [13]:
# 디텍션하는 메인 클래스
class Detector:
    def __init__(self,prototxtPath, weightsPath, model_path):
        # 얼굴 디텍션 모델 로드
        #prototxt는 네트워크 구성, weights는 가중치
        self.net = cv2.dnn.readNet(prototxtPath, weightsPath)
        
        # 마스크 분류 모델 로드
        self.model = load_model(model_path)
    
    # https://mslilsunshine.tistory.com/70    
    def detect(self, image, _confidence):
        # 이미지 크기 받아오기
        # slicing을 통해 height와 width를 h,w에 받음
        (h, w) = image.shape[:2]
        
        # mean subtraction은 RGB값의 일부를 제외해서 dnn이 분석하기 쉽게 단순화 해주는것.
        # 104,177,123은 mean subtraction의 경험적 최적값.
        # 이미지 300, 300을 변경하고 mean subtraction할 값(104, 177, 123)
        # 참고한 사이트 https://www.inflearn.com/questions/29011
        # cv2.dnn.blobFromImage를 통해 입력영상을 블롭객체로 만듬
        # 사용하는 가중치 모델이 300x300 이므로 이미지 사이즈를 300x300 설정
        # blob은 caffee에서 데이터를 저장하고 소통하는데 사용
        blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
        
        # blob 전달하고 detection 얻음
        self.net.setInput(blob)
        # forward()는 순방향 추론으로, 이미지에서 얼굴 분석하고 값을 detections에 넣음
        detections = self.net.forward()
        
        faces = []
        boxes = []
        
        # detections 는 퍼센티지로 표현한 4차원 배열
        for i in range(0, detections.shape[2]):
            # 정확도 추출
            confidence = detections[0, 0, i, 2]

            # 정확도가 낮으면 거름
            if confidence > _confidence:
                # detections[0,0,i,3]는 전체 폭 중 시작점의 x좌표 상대위치(왼쪽 맨 위 시작점)
                # detections[0,0,i,4]는 전체 높이 중 박스 시작점의 y좌표 상대위치
                # detections[0,0,i,5]는 전체 폭 중 박스 끝점의 x좌표 상대위치  (오른쪽 맨 아래 끝점)
                # detections[0,0,i,6]는 전체 높이 중 박스 끝점의 y좌표 상대위치
                # 300, 300 박스 위치에 원본크기를 곱하여 실제 박스위치를 가져옴
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                
                (startX, startY, endX, endY) = box.astype("int")

                # 박스 위치가 이미지를 넘어가면 이미지 바운더리에 닿도록 설정
                (startX, startY) = (max(0, startX), max(0, startY))
                (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

                # 얼굴에서 이미지 부분만 가져옴
                face = image[startY:endY, startX:endX]
                # 이미지에서 얼굴이 안잘리면 넘어감
                if len(face) == 0:
                    continue
                # 얼굴 분류하는 모델의 INPUT은 RGB이고, CV를 거친 데이터포맷은 BGR이므로 RGB로 변경
                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                # 크기를 잘린 이미지를 160, 160으로 줄임
                face = cv2.resize(face, (160, 160))
                # cv mat를 numpy array로 변경
                face = img_to_array(face)
                # 분류를 하기위한 preprocessing 진행
                face = preprocess_input(face)
                # 배치로 한번에 계산하기위해 우선 list로 담아둠
                faces.append(face)
                boxes.append([box,w,h])
        # 얼굴이 검출되지 않으면 none 리턴
        if len(faces) ==0:
            return None
        else:
            # process_image 한 값을 리턴
            return self.process_image(image, faces, boxes)
        
    def process_image(self, image, faces, boxes):
        # 이미지 분류 모델 실행
        outputs = self.model.predict(np.array(faces))

        for index, output in enumerate(outputs):
            # 분류모델의 결과는 MASK의 정확도, WITHOUTMASK의 정확도
            (mask, withoutMask) = output

            # 이전에 저장했던 박스
            box,w,h = boxes[index]
            (startX, startY, endX, endY) = box.astype("int")
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # MASK 정확도가 높으면 MASK 라벨을 붙이고 반대면 No Mask 라벨을 붙임
            if mask > withoutMask:
                label = "Mask"
            else:
                label = "No Mask-mosaic"
            # 라벨이 MASK일때, 박스
            if label == "Mask":
                label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
                # 이미지 위에 글씨 넣기
                cv2.putText(image, label, (startX, startY - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                # 이미지에 박스치기
                cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
            # 라벨이 No MASK일때 박스, 모자이크
            if label == "No Mask-mosaic":
                label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
                # 이미지 위에 글씨 넣기
                cv2.putText(image, label, (startX, startY - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                # 이미지에 박스치기
                cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
                # 모자이크 처리
                image[startY:endY, startX:endX] = self.mosaic(image[startY:endY, startX:endX])
        return image

    # 일정 부분의 이미지를 작게했다 원본으로 변경 -> 모자이크
    def mosaic(self,src, ratio=0.1):
        small = cv2.resize(src, None, fx=ratio, fy=ratio,
                           interpolation=cv2.INTER_NEAREST)
        return cv2.resize(small, src.shape[:2][::-1],
                          interpolation=cv2.INTER_NEAREST)


In [14]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-f", "--face", type=str,
                default="face_detector",
                help="face detection model")
# 마스크 노마스크 분류 모델 위치
ap.add_argument("-m", "--model", type=str,
                default="mask_detector_160x160.model",
                help="mask classification model")
# 얼굴 디텍션 정확도
ap.add_argument("-c", "--confidence", type=float,
                default=0.5,
                help="set minimum confidence")
args = vars(ap.parse_args(args=[]))

In [15]:
# 얼굴 디텍션 모델 위치.
print("얼굴 인식 모델 로드")
prototxtPath = os.path.sep.join([args['face'],
                                 "deploy.prototxt"])
weightsPath = os.path.sep.join([args['face'],
                                "res10_300x300_ssd_iter_140000.caffemodel"])

# 분류모델 
print("마스크 분류 모델 로드")
model_path = args["model"]

detector = Detector(prototxtPath, weightsPath, model_path)

#vid = 'Coronavirus - 34803.mp4'

# 웹캠 불러오기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("카메라 연결 실패. 종료")
    exit(0)
while cap.isOpened():
    # 카메라에서 frame 읽어오기
    ret, frame = cap.read()
    if ret:
        # 얼굴 디텍션 + mask, nomask 분류 결과 이미지로 받기
        im_to_show = detector.detect(frame, args['confidence'])
        # 얼굴이 디텍션이 안된 경우, 원본이미지를 넣음
        if im_to_show is None:
            im_to_show = frame

        # 이미지 보여주기
        cv2.imshow("img", im_to_show)
        key = cv2.waitKey(1)
        # 27은 ESC 키값, ESC누르면 프로그램 정지
        if key == 27:
            break
            
cv2.waitKey(0)
cv2.destroyAllWindows

얼굴 인식 모델 로드
마스크 분류 모델 로드


<function destroyAllWindows>